In [1]:
import obspy_local
import obspy
from obspy_local.obspy_local.io.segy.core import _read_segy
import glob
from scipy.signal import spectrogram, butter, sosfilt, decimate, detrend
from scipy.signal.windows import hann
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import os
import types
import copy
import h5py

In [21]:
'''

Read and downsample the VSP files

'''

f = "borehole_data/VSP/RESPONDER_Jul2019_L028_300mSouth.sgy"
st = _read_segy(f,npts=2799)

In [22]:
st

1046 Trace(s) in Stream:

Seq. No. in line:  318 | 2019-07-07T12:14:03.000000Z - 2019-07-07T12:14:03.699500Z | 4000.0 Hz, 2799 samples
...
(1044 other traces)
...
Seq. No. in line: 1363 | 2019-07-07T12:14:03.000000Z - 2019-07-07T12:14:03.699500Z | 4000.0 Hz, 2799 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [ ]:
# define function for converting files
def downsample_file(f, out_path, nx_old, nx_new, fs_new):
    try:
        st = _read_segy(f,npts=nx_old)

        # resample
        st = st.resample(fs_new)

        # correct metadata
        st.stats.binary_file_header.number_of_samples_per_data_trace = nx_new
        st.stats.binary_file_header.sample_interval_in_microseconds = int(1/fs_new*1e6)

        # convert to float32
        for tr in st:
            tr.data = tr.data.astype('float32')

        # write file
        st.write(out_path+f.split("/")[-1],format="SEGY")
    except:
        print("Issue processing file " + f)

In [ ]:
# list all the 4khz Greenland files
path_4khz = "/1-fnp/petasaur/p-wd03/greenland/Store Glacier DAS data/"
files_4khz = glob.glob(path_4khz+"4kHz/*")

# remove any previously-processed files
out_path = "/1-fnp/pnwstore1/p-wd05/greenland/resampled/"
out_files = glob.glob(out_path+"*")
out_files = [f.split("/")[-1] for f in out_files]

In [ ]:
'''

Read 4khz segy files using obspy, downsample, and write

'''

# set parameters
fs_new = 1000
nx_new = 30000
nx_old = 120000

# get list of files to process
arg_list = []
for f in files_4khz:
    if f.split("/")[-1] not in out_files:
        arg_list.append(f)
        
# run in serial
for f in arg_list:
    downsample_file(f, out_path, nx_old, nx_new, fs_new)

In [ ]:
# list all the natively 1khz files
path_1khz = "/1-fnp/petasaur/p-wd03/greenland/Store Glacier DAS data/1kHz/*"
path_resampled = "/1-fnp/pnwstore1/p-wd05/greenland/data/resampled/*"
files_1khz = glob.glob(path_1khz)

# list all the resamopled 1khz files
files_resampled = glob.glob(path_resampled)

# combine and sort
files_1khz_all = files_1khz + files_resampled
files_1khz_all.sort()

# set output path
out_path = "/fd1/solinger/"

In [ ]:
'''

Combine files into mseed files for each channel for the entire dataset

'''

# choose which channels to write continous files for
start_channel = 331
end_channel = 1362
channels = np.arange(start_channel,end_channel,10)

# make empty stream to fill
st = obspy.Stream()

# iterate through each 30 second file
for f in files_1khz_all:
    try:
        # read in all channels for current file
        st_tmp = _read_segy(f)

        # add each channel to cumulative stream
        for channel in channels:
            st_tmp[channel].stats.station = str(channel)
            st += st_tmp[channel]
    except:
        print("Issue processing file " + f)

# merge into one stream 
st.merge(fill_value=0)

# write a separate file for each channel
for channel in channels:
    st_channel = st.select(station=str(channel))
    st_channel.write(out_path + "channel_"+str(channel)+".mseed",format="MSEED")

In [9]:
def read_data(file,channels):
    st = _read_segy(file)
    t = st[0].times(type="utcdatetime")
    data = np.zeros((st[0].stats.npts,len(channels)))
    for i,channel in enumerate(channels):
        data[:,i] = st[channel].data
    return data,t


def design_butter(lowcut, highcut, fs, order=4):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = butter(order, [low, high], analog=False, btype='band', output='sos')
        return sos

    
def filter_butter(data, lowcut, highcut, fs, order=4):
        sos = design_butter(lowcut, highcut, fs, order=order)
        y = sosfilt(sos, data, axis=0)
        return y
    
    
def taper(data,taper_length,fs):
    taper_length = int(taper_length*2*fs)
    window_ends = hann(taper_length*2)
    window = np.ones(data.shape)
    for i in range(window.shape[1]):
        window[0:taper_length,i] = window_ends[0:taper_length]
        window[-taper_length:window.shape[0],i] = window_ends[-taper_length:window.shape[0]]
    data = window * data
    return data


def get_data(st,s):
    data = np.zeros((s.file_length*s.fs,len(s.channels)))
    for i,channel in enumerate(s.channels):
        data[:,i] = st.select(station=str(channel))[0].data
    return data
    

def save_data_h5(s):
    
    # get start and endtime for saving
    startstr = s.t[0].datetime.strftime("%Y-%m-%dT%H:%M:%S")
    endstr = s.t[-1].datetime.strftime("%Y-%m-%dT%H:%M:%S")
    timestamps = [time.datetime.timestamp() for time in s.t]
    
    # save result for this file
    fname = (s.out_path+"chans"+str(s.channels[0])+"-"+str(s.channels[-1])+
            "_fs"+str(int(s.fs_new))+"_"+startstr+"-"+endstr+".h5")
    
    # save array of data to h5
    with h5py.File(fname,"w") as f:
        f["data"] = s.data
        f["timestamps"] = timestamps
        f.attrs["starttime"] = s.t[0].datetime.strftime("%Y-%m-%dT%H:%M:%S%f")
        f.attrs["endtime"] = s.t[-1].datetime.strftime("%Y-%m-%dT%H:%M:%S%f")
    f.close()
    
    
def read_and_preprocess(s):
    try:
        # read the file and preprocess
        data,s.t = read_data(s.file,s.channels)
        data = preprocess_data(data,s)
        s.data = data
    except:
        print("Issue processing file " + f)
        s.data = []
        s.t = []
        
    return s


def preprocess_data(data,s):
    data = detrend(data, axis=0, type='linear')
    data = taper(data,s.taper_length,s.fs)
    sos = design_butter(s.freq[0],s.freq[1],s.fs)
    data = filter_butter(data,s.freq[0],s.freq[1],s.fs)
    s.fs_new = s.freq[1]*2
    data = decimate(data,int(s.fs/s.fs_new),axis=0)
    return data            

In [ ]:
'''

Save the entire preprocessed dataset as h5 files

FIGURE OUT WHY THIS DOESNT EXECUTE PROPERLY WHEN DEFINED IN MODULE!
RUNS BUT DOESN'T ERROR WHEN TRYING TO READ BAD FILE WHEN...
    1. EVERYTHING IN MODULE WITHOUT __name__ ==__'main'__ 
    2. FUNCTIONS DEFINED IN MODULE WITH imap_unordered IN __name__ ==__'main'__ IN NOTEBOOK
RUNS PROPERLY WHEN EVERYTHING IS DEFINED IN NOTEBOOK WITH imap_unordered IN __name__ ==__'main'__

'''

# list all the files
path_1khz = "/1-fnp/petasaur/p-wd03/greenland/Store Glacier DAS data/1kHz/*"
path_resampled = "/1-fnp/pnwstore1/p-wd05/greenland/data/resampled/*"
files_1khz = glob.glob(path_1khz)
files_resampled = glob.glob(path_resampled)
files_1khz_all = files_1khz + files_resampled
files_1khz_all.sort() 

# set save file saving parameters
s = types.SimpleNamespace()
s.channels = range(331,1361)
s.file_length = 30
s.duration = 300
s.fs = 1000
s.freq = [10,100]
s.taper_length = 0.25
s.out_path = "/store_ssd4/greenland/data/"
s.files = files_1khz_all[7340:]
s.n_procs = 24

# save the dataset as h5 files
#save_dataset_h5(s)

# read the relevant files, fill metadata, taper, and filter
inputs = []
for f in s.files:

    # add to preprocessing input list
    s.file = f
    inputs.append(copy.deepcopy(s))

# map inputs to read_preprocess_save as each call finishes
if __name__ == '__main__':
    multiprocessing.freeze_support()
    p = multiprocessing.Pool(processes=s.n_procs)
    for result in p.imap_unordered(read_and_preprocess,inputs):
        if len(result.data) != 0:
            save_data_h5(result)
    p.close()
    p.join()    